In [ ]:
# Third-party
import astropy.coordinates as coord
from astropy.table import Table
import astropy.units as u
from astropy.constants import G
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from scipy.stats import binned_statistic
import gala.coordinates as gc
from pyia import GaiaData

from matplotlib.patches import Circle
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from gap import *

In [ ]:
gd1 = GaiaData('/Users/adrian/projects/gd1-dr2/data/gd1-master.fits')

In [ ]:
iso = Table.read('/Users/adrian/data/Isochrones/PARSEC/FeH_-1.4_iso.fits')
iso_mask = (iso['stage']>=0) & (iso['stage']<3) & (iso['log(age/yr)'] == 10)
iso = iso[iso_mask]

iso_g = iso['ps1_g'] + coord.Distance(8*u.kpc).distmod.value

## What is the linear mass density of stars along GD-1?

In [ ]:
stream = gd1[gd1.pm_mask & gd1.gi_cmd_mask]

segment_mask = (stream.phi1 > -40*u.deg) & (stream.phi1 < -30*u.deg)
main = stream[segment_mask & (np.abs(stream.phi2) < 0.5*u.deg)]
spur = stream[segment_mask & (stream.phi2 > 0.5*u.deg) & (stream.phi2 < 2.*u.deg)]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))

ax.plot(stream.phi1, stream.phi2,
        marker='.', color='k', alpha=0.5, ls='none')

ax.set_xlim(-75, -10)
ax.set_ylim(-10, 10)

ax.set_xlabel(r'$\phi_1$ [deg]')
ax.set_ylabel(r'$\phi_2$ [deg]')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))

ax.plot(stream.phi1, stream.phi2,
        marker='.', color='#aaaaaa', alpha=0.5, ls='none')

ax.plot(main.phi1, main.phi2,
        marker='.', color='k', alpha=0.8, ls='none')

ax.plot(spur.phi1, spur.phi2,
        marker='.', color='tab:red', alpha=0.8, ls='none')

ax.set_xlim(-55, -25)
ax.set_ylim(-10, 10)

ax.set_xlabel(r'$\phi_1$ [deg]')
ax.set_ylabel(r'$\phi_2$ [deg]')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

ax.plot(main.g0 - main.i0, main.g0,
        marker='.', ls='none', alpha=0.5)
ax.set_xlim(-1, 2)
ax.set_ylim(21.5, 14)
fig.tight_layout()

In [ ]:
gd1_lf_mask = (iso_g > 18) & (iso_g < 21)
gd1_lf_iso = iso[gd1_lf_mask]

nstream = ((main.g0 > 18*u.mag) & (main.g0 < 21*u.mag)).sum()
nspur = ((spur.g0 > 18*u.mag) & (spur.g0 < 21*u.mag)).sum()
nstream, nspur

In [ ]:
fac = np.sum((gd1_lf_iso['int_IMF'][1:] - gd1_lf_iso['int_IMF'][:-1]) / u.Msun)
M_stream = nstream / fac
M_spur = nspur / fac

stream_linear_dens = M_stream / (10*u.deg)
spur_linear_dens = M_spur / (10*u.deg)
stream_linear_dens, spur_linear_dens

## Select an existing HSC field at comparable Galactic latitude

In [ ]:
hsc = Table.read('/Users/adrian/data/HSC/hsc-stars-gri.fits')

In [ ]:
c = coord.SkyCoord(ra=hsc['ra']*u.deg, dec=hsc['dec']*u.deg)

In [ ]:
gal = c.transform_to(coord.Galactic)

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.plot(gal.l.degree[::64],
        gal.b.degree[::64], 
        marker='.', ls='none')
ax.axhline(52)
ax.axhline(-52)
ax.axvline(188)
# ax.set_xlim(0, 360)
ax.set_xlim(73, 78)
ax.set_ylim(-55, -50)

In [ ]:
fr = coord.SkyOffsetFrame(origin=coord.Galactic(l=75*u.deg, b=-52*u.deg))
c_fr = c.transform_to(fr)

# mask = np.sqrt(c_fr.lon.wrap_at(180*u.deg)**2 + c_fr.lat**2) < 1.5*u.deg
mask = np.abs(c_fr.lon.wrap_at(180*u.deg)) < 2.25*u.deg
mask &= np.abs(c_fr.lat) < 0.75*u.deg
print(mask.sum())

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.plot(c_fr.lon.wrap_at(180*u.deg).degree,
        c_fr.lat.degree,
        marker='.', ls='none')
ax.plot(c_fr.lon.wrap_at(180*u.deg).degree[mask],
        c_fr.lat.degree[mask],
        marker='.', ls='none')
ax.set_xlim(-2.25, 2.25)
ax.set_ylim(-1.5, 1.5)

### Over-plot GD-1 isochrone

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,6))

ax.plot(hsc['g_psfflux_mag'][mask] - hsc['i_psfflux_mag'][mask],
        hsc['g_psfflux_mag'][mask],
        marker='.', ls='none', color='k',
        alpha=0.5)

m_g = iso['ps1_g'] + coord.Distance(8*u.kpc).distmod.value
ax.plot(iso['ps1_g']-iso['ps1_i'], m_g,
        marker='', lw=3, color='tab:orange', alpha=0.8)

ax.set_xlim(-1, 3)
ax.set_ylim(26., 19)
ax.set_xlabel('$g-i$')
ax.set_ylabel('$g$')
ax.axhline(24, color='#666666', zorder=-100)
fig.set_facecolor('w')

### Use isochrone to estimate number of stars to sample (following mass function) in the field:

In [ ]:
lf_mask = (m_g > 19.5) & (m_g < 26.2)
lf_iso = iso[lf_mask]

In [ ]:
M_stream = stream_linear_dens * 3*u.deg # mass in GD-1 stream stars in 1 field
M_spur = spur_linear_dens * 3*u.deg # mass in GD-1 spur stars in 1 field
(((lf_iso['int_IMF'][1:] - lf_iso['int_IMF'][:-1]) / u.Msun * M_stream).sum(),
 ((lf_iso['int_IMF'][1:] - lf_iso['int_IMF'][:-1]) / u.Msun * M_spur).sum())

In [ ]:
from scipy.interpolate import interp1d
interp_f = interp1d(lf_iso['int_IMF'], lf_iso['M_act'])
stream_masses = interp_f(np.random.uniform(lf_iso['int_IMF'].min(), lf_iso['int_IMF'].max(), 1000))
spur_masses = interp_f(np.random.uniform(lf_iso['int_IMF'].min(), lf_iso['int_IMF'].max(), 580))
stream_masses.sum(), spur_masses.sum()

In [ ]:
mg_f = interp1d(lf_iso['M_act'], m_g[lf_mask], kind='cubic')
gi_f = interp1d(m_g[lf_mask], (iso['ps1_g']-iso['ps1_i'])[lf_mask], kind='cubic')

stream_sim_mg = mg_f(stream_masses)
base_stream_sim_gi = gi_f(stream_sim_mg)

We simulate 3 cases: limiting mag's of 22.5, 25, 25.5

In [ ]:
glims = [22.5, 24, 25.5]
shifts = [3., 1.9, 0.]
poly_facs = [3, 1.5, 0.]

In [ ]:
plt.plot(hsc['g_psfflux_mag'][mask],
        np.sqrt(hsc['g_psfflux_magsigma'][mask]**2+
                hsc['i_psfflux_magsigma'][mask]**2),
        marker='.', ls='none', alpha=0.2)
plt.xlim(20, 26)
plt.ylim(1e-3, 1e0)
plt.yscale('log')

stream_sim_gi_err = dict()
stream_sim_gi = dict()

for glim, shift in zip(glims, shifts):
    print(glim, shift)
    stream_sim_gi_err[glim] = 10**((-0.8 - -3) / (26 - 20) * (stream_sim_mg - 20 + shift) + -3)
    plt.plot(stream_sim_mg, stream_sim_gi_err[glim], 
             marker='.', ls='none', color='k')

    stream_sim_gi[glim] = np.random.normal(base_stream_sim_gi, 
                                           np.sqrt(stream_sim_gi_err[glim]**2 + 0.025**2), 
                                           size=len(stream_sim_mg))
    
    stream_sim_gi[glim][stream_sim_mg > glim] = np.nan

plt.axvline(24)
plt.axhline(0.15)

plt.axvline(22)
plt.axhline(0.08)

In [ ]:
i_g = iso['ps1_g'] + coord.Distance(8*u.kpc).distmod.value
i_gi = iso['ps1_g'] - iso['ps1_i']

# upsample isochrone:
i_g_up = np.linspace(i_g.min(), i_g.max(), 1024)
i_gi_up = interp1d(i_g, i_gi)(i_g_up)
i_g = i_g_up
i_gi = i_gi_up

iso_polys = dict()
cmd_paths = dict()

for glim, polyfac in zip(glims, poly_facs):
    i_left = i_gi - 0.4*(i_g/(27 - polyfac))**5
    i_right = i_gi + 0.4*(i_g/(27-polyfac))**5

    poly = np.hstack([np.array([i_left, i_g - 0.5/2.]), 
                      np.array([i_right[::-1], i_g[::-1] + 0.5/2.])]).T
    ind = (poly[:,1] < glim) & (poly[:,1] > 20)
    iso_polys[glim] = poly[ind]
    cmd_paths[glim] = mpl.path.Path(iso_polys[glim])

### Plot HSC background and simulated stream for 3 different surveys:

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15, 6),
                         sharex=True, sharey=True)

ax = axes[0]
ax.plot(hsc['g_psfflux_mag'][mask] - hsc['i_psfflux_mag'][mask],
        hsc['g_psfflux_mag'][mask],
        marker='.', ls='none', color='k',
        alpha=0.5)

for ax, glim in zip(axes[1:], glims):    
    ax.plot(stream_sim_gi[glim],
            stream_sim_mg,
            marker='.', ls='none', color='k',
            alpha=0.5)
    
    poly_main = iso_polys[glim]
    ax.plot(poly_main[:, 0], poly_main[:, 1], 
            color='tab:red')

ax.set_xlim(-1, 3)
ax.set_ylim(26., 19)
axes[0].set_xlabel('$g-i$')
axes[1].set_xlabel('$g-i$')
axes[0].set_ylabel('$g$')
# ax.axhline(24, color='#666666', zorder=-100)
fig.set_facecolor('w')

axes[0].set_title('HSC background')
axes[1].set_title('PS1')
axes[2].set_title('Megacam')
axes[3].set_title('HSC')

fig.tight_layout()

In [ ]:
from scipy.optimize import root

In [ ]:
bg_colmag = np.vstack((hsc['g_psfflux_mag'][mask] - hsc['i_psfflux_mag'][mask],
                       hsc['g_psfflux_mag'][mask])).T

SNs = dict()
for glim in glims:
    cmd_path_main = cmd_paths[glim]
    
    fg_stream_colmag = np.vstack((stream_sim_gi[glim], 
                                  stream_sim_mg)).T
    
    bg_iso_mask = cmd_path_main.contains_points(bg_colmag)
    fg_stream_iso_mask = cmd_path_main.contains_points(fg_stream_colmag)
    
    n_bg = bg_iso_mask.sum() / 1.5 # to account for larger bg area
    n_stream = fg_stream_iso_mask.sum()
    
    print(glim, n_bg, n_stream)
    
    SNs[glim] = n_stream / np.sqrt(n_bg)
    print(SNs[glim])
    
    def func(f):
        return (f-1) / (np.sqrt(n_bg) / (f**2 * n_stream) * np.sqrt(2*(1+f**2) + f*(1+f)*n_stream/n_bg)) - 3.
    res = root(func, 1.2)
    print(res.x[0])
    print()
    # break

# Proposal figure

In [ ]:
fig = plt.figure(figsize=(9, 6))

gs = GridSpec(nrows=4, ncols=3)
axes = [fig.add_subplot(gs[:2, 0]), 
        fig.add_subplot(gs[:2, 1]), 
        fig.add_subplot(gs[:2, 2]), 
        fig.add_subplot(gs[2:, :])]

# ------------------------------------------
# Lower panel first
#
ax = axes[-1]
ax.plot(stream.phi1, stream.phi2, 
        color='k', marker='o', ms=2.5, 
        alpha=0.7, mec='none', ls='none')

h = 2.75
r = 1.5
kw = dict(radius=r, alpha=0.8, edgecolor='tab:red', 
          facecolor='none', linewidth=2)

y0 = 0.
y1 = 0.4
xs = np.arange(-47, -13+1e-3, h)

field_cens = []
for x in xs:
    
    if x < -38 or x > -28:
        circ = Circle((x, y0), **kw)
        ax.add_patch(circ)
        field_cens.append([x, y0])
    
    else:
        circ = Circle((x, y1), **kw)
        ax.add_patch(circ)
        field_cens.append([x, y1])
    
n_fields = len(field_cens)
# ax.set_title('GD-1: {0} fields'.format(n_fields))

ax.set_xlabel('$\phi_1$ [deg]')
ax.set_ylabel('$\phi_2$ [deg]')

ax.set_xlim(-52, -8)
ax.set_ylim(-5, 5)

ax.set_aspect('equal')

# ------------------------------------------
# Upper left panel
#
ax = axes[0]
ax.plot(hsc['g_psfflux_mag'][mask] - hsc['i_psfflux_mag'][mask],
        hsc['g_psfflux_mag'][mask],
        marker='o', ms=1.5, ls='none', 
        color='k', alpha=0.5)

glim = 25.5
ax.plot(stream_sim_gi[glim],
        stream_sim_mg,
        marker='o', ms=1.5, ls='none', 
        color='k', alpha=0.5)

ax.set_xlim(-0.5, 3)
ax.set_ylim(26., 19)

ax.xaxis.set_ticks([0, 1, 2])

ax.set_xlabel('$g-i$')
ax.set_ylabel('$g$')

# ------------------------------------------
# Upper middle panel
#
ax = axes[1]

ax.scatter(glims, [SNs[glim] for glim in glims], 
           marker='s', color='k', s=40)

ax.text(glims[0], SNs[glims[0]]-2, 'PS1', 
        fontsize=14, ha='center', va='top')
ax.text(glims[1], SNs[glims[1]]-2, 'Megacam', 
        fontsize=14, ha='center', va='top')
ax.text(glims[2], SNs[glims[2]]-2, 'HSC\nproposed', 
        fontsize=14, ha='center', va='top')

ax.set_xlabel(r'10$\sigma$ $g$')
ax.set_ylabel('cumulative S/N')

ax.set_xticks(np.arange(20, 26+1e-3, 1))
ax.set_xlim(21.5, 26.5)

ax.set_ylim(0, 35)

# ------------------------------------------
# Upper right panel
# see: Gap-profile.ipynb for parameter exploration
ax = axes[2]

gamm = np.sqrt(2)
r0 = 30 * u.kpc
vy = 150 * u.km/u.s
b = 1 * u.kpc
M = 1e7 * u.Msun
alph = 30 * u.deg
t = 100. * u.Myr

wx = 50 * u.km/u.s
wy = 150 * u.km/u.s
wz = 0 * u.km/u.s

wper = np.sqrt(wx**2 + wz**2)
wpar = vy - wy
rs = 250 * u.pc
w = np.sqrt(wpar**2 + wper**2)
tau = (w * r0**2 / (2 * G * M)).to(u.Myr)
psi = np.linspace(-30, 30, 1024)*u.deg

for alph, style in zip([90, 30, 120]*u.deg,
                       [dict(ls='-', lw=3, color='k'),
                        dict(ls='--', lw=2, color='#666666'),
                        dict(ls=':', lw=2, color='#666666')]):
    # for t in np.linspace(100, 900, 16)*u.Myr:
    for t in [600] * u.Myr:
        with u.set_enabled_equivalencies(u.dimensionless_angles()):
            _f = f(t, tau, gamm, vy, r0, wpar, wper, alph).decompose()
            _g = g(t, tau, gamm, vy, r0, wpar, wper, alph, b).decompose()
            _B2 = B2(b, rs, r0, wpar, wper)
            dens = rho(t, _f, _B2, psi, _g)

        ax.plot(psi, dens, marker='', 
                # label=r'$\alpha = {0:.0f}^\circ$'.format(alph.value), 
                label=r'${0:.0f}^\circ$'.format(alph.value), 
                **style)
        
ax.set_xlim(-9, 9)
ax.set_ylim(0., 1.75)

ax.legend(loc='best', fontsize=10)

ax.set_xlabel(r'$\psi$ [deg]')
ax.set_ylabel(r'$\rho/\rho_0$')


# -----

fig.set_facecolor('w')
fig.tight_layout()

fig.savefig('../figure1.pdf')

In [ ]:
g_exp = 500 * u.s
i_exp = 500 * u.s
(n_fields * (g_exp+i_exp)).to(u.hour)

---

In [ ]:
plt.figure()
_, bins, _ = plt.hist(main.phi2, bins='auto', density=True, alpha=0.5)
plt.hist(np.random.normal(0, 0.17, size=1000), density=True, alpha=0.5)

stream_sim_phi1 = np.random.uniform(-1.5, 1.5, size=len(stream_sim_gi))
stream_sim_phi2 = np.random.normal(0, 0.17, size=len(stream_sim_gi))

spur_sim_phi1 = np.random.uniform(-1.5, 1.5, size=len(spur_sim_gi))
spur_sim_phi2 = np.random.normal(0, 0.5, size=len(spur_sim_gi))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), 
                         sharex=True, sharey=True)

ax = axes[0]
ax.plot(c_fr.lon.wrap_at(180*u.deg).degree[mask][bg_iso_mask],
        c_fr.lat.degree[mask][bg_iso_mask],
        marker='.', ls='none', color='k')

ax.plot(stream_sim_phi1[fg_stream_iso_mask], 
        stream_sim_phi2[fg_stream_iso_mask],
        marker='.', ls='none', color='k')

ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)

ax = axes[1]
ax.plot(c_fr.lon.wrap_at(180*u.deg).degree[mask][bg_iso_mask],
        c_fr.lat.degree[mask][bg_iso_mask],
        marker='.', ls='none', color='k')

ax.plot(spur_sim_phi1[fg_spur_iso_mask], spur_sim_phi2[fg_spur_iso_mask],
        marker='.', ls='none', color='k')

axes[0].set_xlabel('lon [deg]')
axes[0].set_ylabel('lat [deg]')
axes[0].set_title('stream field')
axes[1].set_title('spur field')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,6))

ax.plot(hsc['g_psfflux_mag'][mask] - hsc['i_psfflux_mag'][mask],
        hsc['g_psfflux_mag'][mask] - hsc['z_psfflux_mag'][mask],
        marker=',', ls='none', color='k',
        alpha=0.5)

ax.set_xlim(-1, 3)
ax.set_ylim(-3, 4)
ax.set_xlabel('$g-i$')
ax.set_ylabel('$g-z$')
fig.set_facecolor('w')